## Overview
This code mimics Excel Solver’s algorithm for linear problems, allowing users to apply custom tweaks to influence the optimal solution.

## Import packages

In [5]:
import pandas as pd
import numpy as np

## Load sample data
New York air quality data

In [34]:
df_air = pd.read_csv('https://data.cityofnewyork.us/api/views/c3uy-2p5r/rows.csv?accessType=DOWNLOAD')
df_air.head()

,Unique ID,Indicator ID,Name,Measure,Measure Info,Geo Type Name,Geo Join ID,Geo Place Name,Time Period,Start_Date,Data Value,Message
0,179772,640,Boiler Emissions- Total SO2 Emissions,Number per km2,number,UHF42,409.0,Southeast Queens,2015,01/01/2015,0.3,NaN
1,179785,640,Boiler Emissions- Total SO2 Emissions,Number per km2,number,UHF42,209.0,Bensonhurst - Bay Ridge,2015,01/01/2015,1.2,NaN
2,178540,365,Fine particles (PM 2.5),Mean,mcg/m3,UHF42,209.0,Bensonhurst - Bay Ridge,Annual Average 2012,12/01/2011,8.6,NaN
3,178561,365,Fine particles (PM 2.5),Mean,mcg/m3,UHF42,409.0,Southeast Queens,Annual Average 2012,12/01/2011,8.0,NaN
4,823217,365,Fine particles (PM 2.5),Mean,mcg/m3,UHF42,409.0,Southeast Queens,Summer 2022,06/01/2022,6.1,NaN


# Clean data
I will model the fine particles reading for a geo place name using other location's fine particle reading.  The model coefficients will be calculated using the solver algorithm I created

In [101]:
df_selected = df_air[['Name', 'Geo Place Name', 'Time Period', 'Data Value']]

#we will model Fine particles (PM 2.5)
df_selected = df_selected[df_selected['Name']=='Fine particles (PM 2.5)']

#use only data for Annual Average
df_selected = df_selected[['Annual Average' in x for x in df_selected['Time Period']]]

# check how much data we have
print(len(df_selected))

# check time period if there's consistency in the data
print(df_selected['Time Period'].unique())

# get the count for each geo, then filter only for those with complete data
final_geo_list = df_selected['Geo Place Name'].value_counts()
final_geo_list = list(final_geo_list[final_geo_list==final_geo_list.max()].index)
print(final_geo_list)

# filter the selected data to include only those geo loc with complete data
df_selected = df_selected[df_selected['Geo Place Name'].isin(final_geo_list)]

# clean up the time period column, removing the leading 'Annual Average' string
df_selected['Time Period'] = [x.replace('Annual Average ', '') for x in df_selected['Time Period']]

#remove the name column
df_selected = df_selected.drop(['Name'], axis=1)

#show head after cleaning
df_selected.head()

1974
['Annual Average 2012' 'Annual Average 2013' 'Annual Average 2017'
 'Annual Average 2016' 'Annual Average 2021' 'Annual Average 2018'
 'Annual Average 2009' 'Annual Average 2015' 'Annual Average 2011'
 'Annual Average 2020' 'Annual Average 2019' 'Annual Average 2022'
 'Annual Average 2010' 'Annual Average 2014']
['Bensonhurst - Bay Ridge', 'Greenpoint', 'East Flatbush - Flatbush', 'Northeast Bronx', 'Bedford Stuyvesant - Crown Heights', 'Ridgewood - Forest Hills', 'Central Harlem - Morningside Heights', 'West Queens', 'Sunset Park', 'Fordham - Bronx Pk', 'Long Island City - Astoria', 'Southeast Queens', 'Pelham - Throgs Neck', 'Flushing - Clearview', 'Kingsbridge - Riverdale', 'Borough Park', 'East Harlem', 'Washington Heights', 'Canarsie - Flatlands', 'Coney Island - Sheepshead Bay', 'Southwest Queens', 'Rockaways', 'Williamsburg - Bushwick', 'Upper West Side', 'East New York', 'Jamaica', 'Downtown - Heights - Slope']


,Geo Place Name,Time Period,Data Value
2,Bensonhurst - Bay Ridge,2012,8.6
3,Southeast Queens,2012,8.0
10,Bensonhurst - Bay Ridge,2013,8.2
11,Coney Island - Sheepshead Bay,2013,8.1
12,Southeast Queens,2017,6.8


# Pivot Data
make the geo place name features and time period as index

In [139]:
df_pivot = pd.pivot_table(df_selected, values='Data Value', index='Time Period', columns=['Geo Place Name']).reset_index(drop=False)
df_pivot.head()

Geo Place Name,Time Period,Bedford Stuyvesant - Crown Heights,Bensonhurst - Bay Ridge,Borough Park,Canarsie - Flatlands,Central Harlem - Morningside Heights,Coney Island - Sheepshead Bay,Downtown - Heights - Slope,East Flatbush - Flatbush,East Harlem,...,Pelham - Throgs Neck,Ridgewood - Forest Hills,Rockaways,Southeast Queens,Southwest Queens,Sunset Park,Upper West Side,Washington Heights,West Queens,Williamsburg - Bushwick
0,2009,10.8,9.9,10.3,9.9,11.6,9.6,11.5,10.5,11.6,...,10.6,10.4,8.8,9.4,10.0,11.0,12.2,11.6,11.0,11.4
1,2010,9.9,9.2,9.6,9.1,10.6,8.9,10.5,9.8,10.4,...,9.7,9.5,8.0,8.7,9.2,10.2,11.0,10.7,9.9,10.3
2,2011,10.3,9.8,10.1,9.6,11.1,9.4,11.1,10.2,11.1,...,10.5,10.0,8.4,9.2,9.4,10.8,11.5,11.1,10.6,10.9
3,2012,9.1,8.6,8.9,8.4,10.0,8.3,9.9,9.0,10.0,...,9.2,8.8,7.4,8.0,8.3,9.6,10.5,9.9,9.5,9.7
4,2013,8.8,8.2,8.5,8.2,9.7,8.1,9.5,8.6,9.7,...,8.7,8.5,7.6,7.9,8.1,9.1,10.4,9.5,9.1,9.4


# Split validation/train periods and target/control geos
Split the pivot into 4 groups, train period to train the model using solver algorithm. Validation period to check how model works.  target is the dependent variable while control are the independent variables